In [2]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5

COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [4]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [5]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  84.99kB
Step 1/8 : FROM python:3.9-slim-buster
3.9-slim-buster: Pulling from library/python

b88d5577: Pulling fs layer 
16e23423: Pulling fs layer 
da260408: Pulling fs layer 
c79126f6: Pulling fs layer 
Digest: sha256:320a7a4250aba4249f458872adecf92eea88dc6abd2d76dc5c0f01cac9b53990[5A
Status: Downloaded newer image for python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/8 : RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5
 ---> Running in f0f2d566f8ed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 24.7 MB/s eta 0:00:00
   

In [6]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [8]:
!aws ecr create-repository --repository-name $ecr_repository

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:654654222480:repository/demo_prophet",
        "registryId": "654654222480",
        "repositoryName": "demo_prophet",
        "repositoryUri": "654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet",
        "createdAt": 1723635381.503,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [9]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [11]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

9a5f9e9b: Preparing 
6c9095dc: Preparing 
62553ec9: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
62553ec9: Pushed   593.3MB/581.8MB2KPushing  18.22MB/28.86MBPushing  309.8MB/581.8MBlatest: digest: sha256:118155978a6c61834cc58b6c37f247034f1fcf1ed7b5fa5833bd9c5613054cae size: 1997
